# Scrape web
zagor

# libraries

In [1]:
# upgrade pip with python.exe -m pip install --upgrade pip
# open terminal and install with python -m pip install LIBRARY
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import openpyxl
import session_info


c:\mzInstall\Miniconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


# base url

In [2]:
base_url = "https://projekt-spoznaj.si/videoposnetki-in-predstavitve/fair/"

# f-ctions

## scrape: titles and dates
User-Agent header that mimics a typical web browser (like Chrome) to avoid being blocked by the server -> able to bypass the 403 error and fetch the pages
by ChatGPT

In [3]:
# Function to scrape lecture titles and dates from a single page
def scrape_page(url):
    # Define headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # Send request and parse the HTML content with headers
    response = requests.get(url, headers=headers)
    
    # Check the response status
    if response.status_code != 200:
        print(f"Failed to fetch {url}, status code: {response.status_code}")
        return pd.DataFrame()  # Return empty DataFrame if page can't be loaded
    
    print(f"Successfully fetched {url}")
    webpage = BeautifulSoup(response.text, 'html.parser')
    
    # Extract lecture titles
    lecture_titles = [title.get_text(strip=True) for title in webpage.find_all('a', class_='aiovg-link-title')]
    
    # Extract dates and clean them
    dates = [re.search(r'\((\d{1,2}\. \d{1,2}\. \d{4})\)', title).group(1) if re.search(r'\((\d{1,2}\. \d{1,2}\. \d{4})\)', title) else None for title in lecture_titles]
    
    # Clean the titles (remove dates)
    cleaned_titles = [re.sub(r'\s*\(\d{1,2}\. \d{1,2}\. \d{4}\)', '', title).strip() for title in lecture_titles]
    
    # Convert to DataFrame
    data = {
        'Date': pd.to_datetime(dates, format='%d. %m. %Y'),
        'Lecture_Title': cleaned_titles
    }
    return pd.DataFrame(data)

# scrape

In [4]:
urls = [base_url, f"{base_url}page/2/"] # for exact links


lectures_df = pd.DataFrame()
for url in urls:
    lectures_df = pd.concat([lectures_df, scrape_page(url)], ignore_index=True)

Successfully fetched https://projekt-spoznaj.si/videoposnetki-in-predstavitve/fair/
Successfully fetched https://projekt-spoznaj.si/videoposnetki-in-predstavitve/fair/page/2/


# write res

In [5]:
output_folder = os.path.join('..', 'output')
output_filename = "Py-lecture-titles-dates.xlsx"

os.makedirs(output_folder, exist_ok=True)

output_file_path = os.path.join(output_folder, output_filename)
lectures_df.to_excel(output_file_path, index=False)

# session info

In [6]:
session_info.show()